Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Do the [Plotly Dash](https://dash.plot.ly/) Tutorial, Parts 1 & 2.
- [ ] Add your own stretch goal(s) !

In [1]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module1')

     |████████████████████████████████| 133kB 2.9MB/s 
     |████████████████████████████████| 7.1MB 34kB/s 
     |████████████████████████████████| 614kB 29.7MB/s 
     |████████████████████████████████| 225kB 38.5MB/s 
     |████████████████████████████████| 768kB 34.8MB/s 
     |████████████████████████████████| 194kB 20.7MB/s 
     |████████████████████████████████| 51kB 21.9MB/s 
     |████████████████████████████████| 51kB 21.6MB/s 
     |████████████████████████████████| 737kB 37.6MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-2.3.0-py2.py3-none-any.whl size=145035 sha256=d155e0b7f9d06560ac88dd3442fe49e6c63b48e7e571550a0ce45bd6ff4d81d8
  Stored in directory: /root/.cache/pip/wheels/ce/c7/f1/dbfef4848ebb048cb1d4a22d1ed0c62d8ff2523747235e19fe
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27084 sha256=db29e050aded6768ae1ba763de3dcc51a5d51af0abc849278660081a4c1c1957
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708

In [2]:
!pip install fiona
!pip install geopandas

     |████████████████████████████████| 17.9MB 2.8MB/s 
  Created wheel for munch: filename=munch-2.3.2-py2.py3-none-any.whl size=6614 sha256=2e95dbe2a74b46e143b6ed335b1644f2f7e4be7bbb13c8b2384ad460feb97ba4
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch
     |████████████████████████████████| 901kB 2.8MB/s 
     |████████████████████████████████| 9.8MB 37.0MB/s 


In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv('../data/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [5]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Example of Neighborhood Lookup

In [6]:
import geopandas
import json
from shapely.geometry import shape, Point

nyc = geopandas.read_file('https://raw.githubusercontent.com/JimKing100/DS-Unit-2-Regression-Classification/master/module2/NYC.geojson')

def neighborhood(long, lat):
  # construct point based on lon/lat returned by geocoder
  point = Point(long, lat)
  n_name = 'Unknown'

# check each polygon to see if it contains the point
  for i in range(0, len(nyc.index)):
    polygon = shape(nyc.loc[i]['geometry'])
    if polygon.contains(point):
      n_name = nyc.loc[i]['ntaname']
      return n_name
    
  return  n_name


longitude = -73.9667
latitude = 40.7947

n_name = neighborhood(longitude, latitude)
print(n_name)



Upper West Side


### Create Train and Test

In [7]:
df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True)
df['created'].describe()

count                   48817
unique                  48148
top       2016-05-14 01:11:03
freq                        3
first     2016-04-01 22:12:41
last      2016-06-29 21:41:47
Name: created, dtype: object

In [8]:
mask = (df['created'] > '2016-03-31') & (df['created'] < '2016-06-01')
train = df.loc[mask]
train['created'].describe()

count                   31844
unique                  31436
top       2016-05-14 01:11:03
freq                        3
first     2016-04-01 22:12:41
last      2016-05-31 23:10:48
Name: created, dtype: object

In [9]:
mask = (df['created'] >= '2016-06-01')
test = df.loc[mask]
test['created'].describe()

count                   16973
unique                  16712
top       2016-06-25 01:30:16
freq                        3
first     2016-06-01 01:10:37
last      2016-06-29 21:41:47
Name: created, dtype: object

### Train - Build Features

In [10]:
train.shape

(31844, 34)

In [11]:
amenities = ['elevator',	'cats_allowed',	'hardwood_floors',	'dogs_allowed',
              'doorman',	'dishwasher',	'no_fee',	'laundry_in_building',	'fitness_center',
              'pre-war',	'laundry_in_unit',	'roof_deck',	'outdoor_space',	'dining_room',
              'high_speed_internet',	'balcony',	'swimming_pool',	'new_construction',
              'terrace',	'exclusive',	'loft',	'garden_patio',	'wheelchair_access',	'common_outdoor_space']
train['no_amenities'] = train[amenities].sum(axis=1)
train.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,no_amenities
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,2.0,4,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3


In [12]:
hoods = train[['longitude', 'latitude']]
print(hoods.shape)
hoods.head()

(31844, 2)


,longitude,latitude
2,-74.0018,40.7388
3,-73.9677,40.7539
4,-73.9493,40.8241
5,-74.0028,40.7429
6,-73.9660,40.8012


In [0]:
#hoods['neighborhood'] = hoods.apply(lambda x: neighborhood(x['longitude'], x['latitude']), axis=1)
#hoods.head()

In [14]:
hoods.shape

(31844, 2)

In [15]:
n_hoods = pd.read_csv('https://raw.githubusercontent.com/JimKing100/DS-Unit-2-Regression-Classification/master/module2/hoods.csv')
n_hoods = n_hoods.rename(columns={'Unnamed: 0': 'old_index'})
n_hoods.head()


,old_index,longitude,latitude,neighborhood
0,2,-74.0018,40.7388,West Village
1,3,-73.9677,40.7539,Turtle Bay-East Midtown
2,4,-73.9493,40.8241,Hamilton Heights
3,5,-74.0028,40.7429,Hudson Yards-Chelsea-Flatiron-Union Square
4,6,-73.9660,40.8012,Morningside Heights


In [16]:
new_train = pd.merge(train, n_hoods, how='left', left_index=True, right_on=['old_index'])
print(new_train.shape)
new_train.head()


(31844, 39)


,bathrooms,bedrooms,created,description,display_address,latitude_x,longitude_x,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,no_amenities,old_index,longitude_y,latitude_y,neighborhood
0,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,-74.0018,40.7388,West Village
1,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,-73.9677,40.7539,Turtle Bay-East Midtown
2,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,-73.9493,40.8241,Hamilton Heights
3,2.0,4,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,-74.0028,40.7429,Hudson Yards-Chelsea-Flatiron-Union Square
4,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,6,-73.9660,40.8012,Morningside Heights


In [0]:
neighborhood_dict = {
 'Midwood': 1,
 'Bedford:': 2,
 'Fordham South': 3,
 'Borough Park': 4,
 'Rugby-Remsen Village': 5,
 'East Flushing': 6,
 'Woodhaven': 7,
 'Madison': 8,
 'Auburndale': 9,
 'Williamsbridge-Olinville': 10,
 'Murray Hill': 11,
 'East Elmhurst': 12,
 'Brownsville': 13,
 'East New York (Pennsylvania Ave)':14,
 'Kensington-Ocean Parkway': 15,
 'Parkchester': 16,
 'Erasmus': 17,
 'Cambria Heights': 18,
 'East Flatbush-Farragut': 19,
 'Ocean Parkway South': 20,
 'Starrett City': 21,
 'Morrisania-Melrose': 22,
 'Elmhurst': 23,
 'East Village': 24,
 'Glen Oaks-Floral Park-New Hyde Park': 25,
 'Longwood': 26,
 'Yorkville': 27,
 'Upper East Side-Carnegie Hill': 28,
 'Windsor Terrace': 29,
 'Hammels-Arverne-Edgemere': 30,
 'Rikers Island': 31,
 'Hunts Point': 31,
 'Jackson Heights': 32,
 'Bath Beach': 33,
 'Old Town-Dongan Hills-South Beach': 34,
 'Flatbush': 35,
 'Melrose South-Mott Haven North': 36,
 'Ocean Hill': 37,
 'Morningside Heights': 38,
 'Soundview-Bruckner': 39,
 'Allerton-Pelham Gardens': 40,
 'Jamaica Estates-Holliswood': 41,
 'Hollis': 42,
 'Flatlands': 43,
 'East New York': 44,
 'Kingsbridge Heights': 45,
 'Springfield Gardens North': 46,
 'Canarsie': 47,
 'Norwood': 48,
 'Manhattanville': 49,
 'West Village': 50,
 'Grasmere-Arrochar-Ft. Wadsworth': 51,
 'Queens Village': 52,
 'Chinatown': 53,
 'Pelham Bay-Country Club-City Island': 54,
 'Woodlawn-Wakefield': 55,
 'Old Astoria': 56,
 'Astoria': 57,
 'Stuyvesant Town-Cooper Village': 58,
 'Dyker Heights': 59,
 'Bensonhurst West': 60,
 'West New Brighton-New Brighton-St. George': 61,
 'New Brighton-Silver Lake': 62,
 'Westerleigh': 63,
 'University Heights-Morris Heights': 64,
 'Bayside-Bayside Hills': 65,
 'Crown Heights North': 66,
 'East Concourse-Concourse Village': 67,
 'North Corona': 68,
 'Cypress Hills-City Line': 69,
 'Kew Gardens Hills': 70,
 'Pomonok-Flushing Heights-Hillcrest': 71,
 'North Side-South Side': 72,
 'Lower East Side': 73,
 'Greenpoint': 74,
 'Spuyten Duyvil-Kingsbridge': 75,
 'Sunset Park East': 76,
 'Marble Hill-Inwood': 77,
 'Homecrest': 78,
 'Washington Heights North': 79,
 'Steinway': 80,
 'Mott Haven-Port Morris': 81,
 'West Brighton': 82,
 'Central Harlem North-Polo Grounds': 83,
 'Queensbridge-Ravenswood-Long Island City': 84,
 'New Dorp-Midland Beach': 85,
 'Van Cortlandt Village': 86,
 'Co-op City': 87,
 'Bay Ridge': 88,
 'Sunset Park West': 89,
 'Fort Greene': 90,
 'SoHo-TriBeCa-Civic Center-Little Italy': 91,
 'Battery Park City-Lower Manhattan': 92,
 'Clinton': 93,
 'Prospect Heights': 94,
 'Baisley Park': 95,
 'South Jamaica': 96,
 'Ozone Park': 97,
 'Georgetown-Marine Park-Bergen Beach-Mill Basin': 98,
 'Brighton Beach': 99,
 'Bensonhurst East': 100,
 'West Farms-Bronx River': 101,
 'Sheepshead Bay-Gerritsen Beach-Manhattan Beach': 102,
 'Westchester-Unionport': 103,
 'Oakwood-Oakwood Beach': 104,
 'Grymes Hill-Clifton-Fox Hills': 105,
 'Park Slope-Gowanus': 106,
 'Stapleton-Rosebank': 107,
 'Fresh Meadows-Utopia': 108,
 'Bellerose': 109,
 'Stuyvesant Heights': 110,
 'East Williamsburg': 111,
 'Ridgewood': 112,
 'East Harlem South': 113,
 'Rego Park': 114,
 'East Harlem North': 115,
 'Bushwick North': 116,
 'Bushwick South': 117,
 'Central Harlem South': 118,
 'College Point': 119,
 'Midtown-Midtown South': 120,
 'Glendale': 121,
 'Charleston-Richmond Valley-Tottenville': 122,
 'park-cemetery-etc-Staten Island': 123,
 'New Springville-Bloomfield-Travis': 124,
 'Todt Hill-Emerson Hill-Heartland Village-Lighthouse Hill': 125,
 'South Ozone Park': 126,
 'Lindenwood-Howard Beach': 127,
 'Prospect Lefferts Gardens-Wingate': 128,
 'Murray Hill-Kips Bay': 129,
 "Mariner's Harbor-Arlington-Port Ivory-Graniteville": 130,
 'Crown Heights South': 131,
 'Brooklyn Heights-Cobble Hill': 132,
 'Port Richmond': 133,
 'Hunters Point-Sunnyside-West Maspeth': 134,
 'Claremont-Bathgate': 135,
 'Van Nest-Morris Park-Westchester Square': 136,
 'Pelham Parkway': 137,
 'Mount Hope': 138,
 'Ft. Totten-Bay Terrace-Clearview': 139,
 'Whitestone': 140,
 'Turtle Bay-East Midtown': 141,
 'Lenox Hill-Roosevelt Island': 142,
 'Elmhurst-Maspeth': 143,
 'Woodside': 144,
 'St. Albans': 145,
 'Laurelton': 146,
 'Hamilton Heights': 147,
 'Jamaica': 148,
 'Richmond Hill': 149,
 'Briarwood-Jamaica Hills': 150,
 'Kew Gardens': 151,
 'Middle Village': 152,
 'Maspeth': 153,
 'Upper West Side': 154,
 'Lincoln Square': 155,
 'Bronxdale': 156,
 'Oakland Gardens': 157,
 'Douglas Manor-Douglaston-Little Neck': 158,
 "Annadale-Huguenot-Prince's Bay-Eltingville": 159,
 'Great Kills': 160,
 'Seagate-Coney Island': 161,
 'Corona': 162,
 'Schuylerville-Throgs Neck-Edgewater Park': 163,
 'Gravesend': 164,
 'East Tremont': 165,
 'North Riverdale-Fieldston-Riverdale': 166,
 'Bedford Park-Fordham North': 167,
 'Belmont': 168,
 'Eastchester-Edenwald-Baychester': 169,
 'Breezy Point-Belle Harbor-Rockaway Park-Broad Channel': 170,
 'Crotona Park East': 171,
 'Rossville-Woodrow':172,
 'Arden Heights': 173,
 'Far Rockaway-Bayswater': 174,
 'Soundview-Castle Hill-Clason Point-Harding Park': 175,
 'park-cemetery-etc-Bronx': 176,
 'Carroll Gardens-Columbia Street-Red Hook': 177,
 'park-cemetery-etc-Manhattan': 178,
 'Rosedale': 179,
 'Flushing': 180,
 'Queensboro Hill': 181,
 'Hudson Yards-Chelsea-Flatiron-Union Square': 182,
 'Gramercy': 183,
 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill': 184,
 'Clinton Hill': 185,
 'Williamsburg': 186,
 'park-cemetery-etc-Brooklyn': 187,
 'Washington Heights South': 188,
 'Highbridge': 189,
 'West Concourse': 190,
 'Forest Hills': 191,
 'park-cemetery-etc-Queens': 192,
 'Springfield Gardens South-Brookville': 193,
 'Airport': 194
}

In [18]:
new_train['nid'] = new_train['neighborhood'].map(neighborhood_dict)
new_train.head()

,bathrooms,bedrooms,created,description,display_address,latitude_x,longitude_x,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,no_amenities,old_index,longitude_y,latitude_y,neighborhood,nid
0,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,-74.0018,40.7388,West Village,50.0
1,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,-73.9677,40.7539,Turtle Bay-East Midtown,141.0
2,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,-73.9493,40.8241,Hamilton Heights,147.0
3,2.0,4,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,-74.0028,40.7429,Hudson Yards-Chelsea-Flatiron-Union Square,182.0
4,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,6,-73.9660,40.8012,Morningside Heights,38.0


In [0]:
values = {'nid': 0}
new_train = new_train.fillna(value=values)

In [20]:
train = new_train
train.shape

(31844, 40)

In [37]:
train['interest_level'].unique()

array(['high', 'low', 'medium'], dtype=object)

In [0]:
interest = {
 'low': 1,
 'medium:': 2,
 'high': 3
}

In [41]:
train['interest'] = train['interest_level'].map(interest)
train.head()

,bathrooms,bedrooms,created,description,display_address,latitude_x,longitude_x,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,no_amenities,old_index,longitude_y,latitude_y,neighborhood,nid,interest
0,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,-74.0018,40.7388,West Village,50.0,3.0
1,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,-73.9677,40.7539,Turtle Bay-East Midtown,141.0,1.0
2,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,-73.9493,40.8241,Hamilton Heights,147.0,1.0
3,2.0,4,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,-74.0028,40.7429,Hudson Yards-Chelsea-Flatiron-Union Square,182.0,NaN
4,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,6,-73.9660,40.8012,Morningside Heights,38.0,1.0


In [0]:
values = {'interest': 0}
train = train.fillna(value=values)

In [46]:
train.shape

(31844, 41)

### Test - Build Features

In [22]:
test.shape

(16973, 34)

In [23]:
amenities = ['elevator',	'cats_allowed',	'hardwood_floors',	'dogs_allowed',
              'doorman',	'dishwasher',	'no_fee',	'laundry_in_building',	'fitness_center',
              'pre-war',	'laundry_in_unit',	'roof_deck',	'outdoor_space',	'dining_room',
              'high_speed_internet',	'balcony',	'swimming_pool',	'new_construction',
              'terrace',	'exclusive',	'loft',	'garden_patio',	'wheelchair_access',	'common_outdoor_space']
test['no_amenities'] = test[amenities].sum(axis=1)
test.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,no_amenities
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
11,1.0,1,2016-06-03 03:21:22,Check out this one bedroom apartment in a grea...,W. 173rd Street,40.8448,-73.9396,1675,644 W. 173rd Street,low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,1.0,1,2016-06-01 03:11:01,Spacious 1-Bedroom to fit King-sized bed comfo...,East 56th St..,40.7584,-73.9648,3050,315 East 56th St..,low,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
24,2.0,4,2016-06-07 04:39:56,SPRAWLING 2 BEDROOM FOUND! ENJOY THE LUXURY OF...,W 18 St.,40.7391,-73.9936,7400,30 W 18 St.,medium,1,1,1,1,1,1,0,0,1,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,11


In [24]:
thoods = test[['longitude', 'latitude']]
print(thoods.shape)
thoods.head()

(16973, 2)


,longitude,latitude
0,-73.9425,40.7145
1,-73.9667,40.7947
11,-73.9396,40.8448
14,-73.9648,40.7584
24,-73.9936,40.7391


In [0]:
#thoods['neighborhood'] = thoods.apply(lambda x: neighborhood(x['longitude'], x['latitude']), axis=1)
#thoods.head()

In [0]:
thoods.to_csv('/content/thoods.csv')

In [27]:
tn_hoods = pd.read_csv('https://raw.githubusercontent.com/JimKing100/DS-Unit-2-Regression-Classification/master/module2/thoods.csv')
tn_hoods = tn_hoods.rename(columns={'Unnamed: 0': 'old_index'})
tn_hoods.head()

,old_index,longitude,latitude,neighborhood
0,0,-73.9425,40.7145,East Williamsburg
1,1,-73.9667,40.7947,Upper West Side
2,11,-73.9396,40.8448,Washington Heights South
3,14,-73.9648,40.7584,Turtle Bay-East Midtown
4,24,-73.9936,40.7391,Hudson Yards-Chelsea-Flatiron-Union Square


In [28]:
new_test = pd.merge(test, tn_hoods, how='left', left_index=True, right_on=['old_index'])
print(new_test.shape)
new_test.head()


(16973, 39)


,bathrooms,bedrooms,created,description,display_address,latitude_x,longitude_x,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,no_amenities,old_index,longitude_y,latitude_y,neighborhood
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-73.9425,40.7145,East Williamsburg
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,-73.9667,40.7947,Upper West Side
2,1.0,1,2016-06-03 03:21:22,Check out this one bedroom apartment in a grea...,W. 173rd Street,40.8448,-73.9396,1675,644 W. 173rd Street,low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,-73.9396,40.8448,Washington Heights South
3,1.0,1,2016-06-01 03:11:01,Spacious 1-Bedroom to fit King-sized bed comfo...,East 56th St..,40.7584,-73.9648,3050,315 East 56th St..,low,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,14,-73.9648,40.7584,Turtle Bay-East Midtown
4,2.0,4,2016-06-07 04:39:56,SPRAWLING 2 BEDROOM FOUND! ENJOY THE LUXURY OF...,W 18 St.,40.7391,-73.9936,7400,30 W 18 St.,medium,1,1,1,1,1,1,0,0,1,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,11,24,-73.9936,40.7391,Hudson Yards-Chelsea-Flatiron-Union Square


In [29]:
new_test['nid'] = new_test['neighborhood'].map(neighborhood_dict)
new_test.head()

,bathrooms,bedrooms,created,description,display_address,latitude_x,longitude_x,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,no_amenities,old_index,longitude_y,latitude_y,neighborhood,nid
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-73.9425,40.7145,East Williamsburg,111.0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,-73.9667,40.7947,Upper West Side,154.0
2,1.0,1,2016-06-03 03:21:22,Check out this one bedroom apartment in a grea...,W. 173rd Street,40.8448,-73.9396,1675,644 W. 173rd Street,low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,-73.9396,40.8448,Washington Heights South,188.0
3,1.0,1,2016-06-01 03:11:01,Spacious 1-Bedroom to fit King-sized bed comfo...,East 56th St..,40.7584,-73.9648,3050,315 East 56th St..,low,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,14,-73.9648,40.7584,Turtle Bay-East Midtown,141.0
4,2.0,4,2016-06-07 04:39:56,SPRAWLING 2 BEDROOM FOUND! ENJOY THE LUXURY OF...,W 18 St.,40.7391,-73.9936,7400,30 W 18 St.,medium,1,1,1,1,1,1,0,0,1,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,11,24,-73.9936,40.7391,Hudson Yards-Chelsea-Flatiron-Union Square,182.0


In [0]:
values = {'nid': 0}
new_test = new_test.fillna(value=values)

In [31]:
test = new_test
test.shape

(16973, 40)

In [43]:
test['interest'] = test['interest_level'].map(interest)
test.head()

,bathrooms,bedrooms,created,description,display_address,latitude_x,longitude_x,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,no_amenities,old_index,longitude_y,latitude_y,neighborhood,nid,interest
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-73.9425,40.7145,East Williamsburg,111.0,NaN
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,-73.9667,40.7947,Upper West Side,154.0,1.0
2,1.0,1,2016-06-03 03:21:22,Check out this one bedroom apartment in a grea...,W. 173rd Street,40.8448,-73.9396,1675,644 W. 173rd Street,low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,-73.9396,40.8448,Washington Heights South,188.0,1.0
3,1.0,1,2016-06-01 03:11:01,Spacious 1-Bedroom to fit King-sized bed comfo...,East 56th St..,40.7584,-73.9648,3050,315 East 56th St..,low,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,14,-73.9648,40.7584,Turtle Bay-East Midtown,141.0,1.0
4,2.0,4,2016-06-07 04:39:56,SPRAWLING 2 BEDROOM FOUND! ENJOY THE LUXURY OF...,W 18 St.,40.7391,-73.9936,7400,30 W 18 St.,medium,1,1,1,1,1,1,0,0,1,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,11,24,-73.9936,40.7391,Hudson Yards-Chelsea-Flatiron-Union Square,182.0,NaN


In [44]:
test.shape

(16973, 41)

In [0]:
values = {'interest': 0}
test = test.fillna(value=values)

In [48]:
test.shape

(16973, 41)

### Run Model

In [0]:
# 1. Import the appropriate estimator class from Scikit-Learn
from sklearn.linear_model import LinearRegression
import numpy as np

# 2. Instantiate this class
model = LinearRegression()

# 3. Arrange X features matrix & y target vector
features = ['bedrooms', 'bathrooms', 'no_amenities', 'nid']
target = 'price'

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

# 4. Fit the model
model.fit(X_train, y_train)

# 5. Apply the model
y_pred = model.predict(X_train)

In [59]:
# Show the coefficient
model.coef_

array([ 3.91940915e+02,  1.93596153e+03,  7.84242539e+01,  2.16372428e-02,
       -6.18154673e+00])

In [60]:
# Show the intercept
model.intercept_

277.33081372267634

### Run Metrics

In [61]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# Print regression metrics
train_mse = mean_squared_error(y_train, y_pred)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(y_train, y_pred)
train_r2 = r2_score(y_train, y_pred)
print('Train Mean Squared Error:', train_mse)
print('Train Root Mean Squared Error:', train_rmse)
print('Train Mean Absolute Error:', train_mae)
print('Train R^2:', train_r2)
print('\n')

ty_pred = model.predict(X_test)
# Print regression metrics
test_mse = mean_squared_error(y_test, ty_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, ty_pred)
test_r2 = r2_score(y_test, ty_pred)
print('Test Mean Squared Error:', test_mse)
print('Test Root Mean Squared Error:', test_rmse)
print('Test Mean Absolute Error:', test_mae)
print('Test R^2:', test_r2)

Train Mean Squared Error: 1449481.0474593542
Train Root Mean Squared Error: 1203.9439552817041
Train Mean Absolute Error: 795.4377900282419
Train R^2: 0.533182648998997


Test Mean Squared Error: 1421050.1911330747
Test Root Mean Squared Error: 1192.078097749084
Test Mean Absolute Error: 799.762367546125
Test R^2: 0.5427797725094312
